# Constructing drive time based service areas
This sample shows how the `network` module of the ArcGIS API for Python can be used to construct service areas. In this sample, we generate service areas for two of the fire stations in central Tokyo, Japan. We later observe how the service area varies by time of day for a fire station in the city of Los Angeles.

### Service areas
A network service area is a region that encompasses all accessible streets (that is, streets that are within a specified impedance). For instance, the 5-minute service area for a point on a network includes all the streets that can be reached within five minutes from that point.

Service areas also help evaluate accessibility. Concentric service areas show how accessibility varies. Once service areas are created, you can use them to identify how much land, how many people, or how much of anything else is within the neighborhood or region.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-0166E6A5-CDEC-4A03-A022-F4A6EB7F9EAD-web.png)

Service area solver provides functionality for finding out how far a vehicle could go within a specified time or distance limit. 

## Connect to the GIS
Establish a connection to your organization which could be an ArcGIS Online organization or an ArcGIS Enterprise. To be able to run the code in this sample notebook, you would need to provide credentials of a user within an ArcGIS Online organization.

In [1]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS
from getpass import getpass

user_name = 'arcgis_python'
password = getpass()
my_gis = GIS('https://www.arcgis.com', user_name, password)

········


### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this sample, since we are creating service areas, we need to create a `ServiceAreaLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this sample, we provide a `serviceArea` URL to create a `ServiceAreaLayer` object. 

Since all ArcGIS Online organizations already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. If you have your own ArcGIS Server based map service with network analysis capability enabled, you would need to provide the URL for this service.

Let us start by importing the `network` module

In [3]:
from arcgis.network import ServiceAreaLayer

In [4]:
service_area_url = my_gis.properties.helperServices.serviceArea.url
service_area_url

'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [5]:
sa_layer = ServiceAreaLayer(service_area_url, gis=my_gis)

## Calculate service area from SD convention center

In [6]:
from arcgis.geocoding import geocode
sd_geocode = geocode('San Diego Convention Center')[0]
sd_geocode['location']

{'x': -117.16097999999994, 'y': 32.70691000000005}

In [7]:
from arcgis.features import Feature, FeatureSet

f1 = Feature(geometry=sd_geocode['location'])
fset = FeatureSet([f1], geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

Let us display the fire stations on a map

In [8]:
map1 = my_gis.map('San Diego, CA', zoomlevel=12)
map1

In [9]:
map1.draw(fset)

## Compute the service area
To compute the service area (area accessible to each facility based on drive times), we use the `solve_service_area()` method of a `ServiceAreaLayer` object. As the fire trucks will be travelling away from the stations, we need to specify the direction of travel in the `travel_direction` parameter. Also, since for the type of vehicles is fire trucks, we could specify the travel mode to make it easier to supply all other related parameters.

In [10]:
travel_modes = sa_layer.retrieve_travel_modes()
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

In [11]:
for t in travel_modes['supportedTravelModes']:
    print(t['name'])

Walking Time
Rural Driving Distance
Driving Time
Driving Distance
Walking Distance
Rural Driving Time
Trucking Time
Trucking Distance


In [12]:
result = sa_layer.solve_service_area(fset, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionFromFacility',
                                     travel_mode=truck_mode)

#### Read the result back as a `FeatureSet`
The `result` variable contains the service area as a dictionary. We inspect its keys and construct `Feature` and `FeatureSet` objects out of it to display in the map

In [13]:
result.keys()

dict_keys(['saPolygons', 'messages'])

In [14]:
result['saPolygons'].keys()

dict_keys(['geometryType', 'spatialReference', 'features', 'fieldAliases'])

In [15]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)

In [16]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

### Visualize the service area on the map
From the DataFrame above, we know, there are 3 service area polygons for each fire station. The drive times are given as a range between `FromBreak` and `ToBreak` columns. Let us use this information to visualize the polygons with different colors and appropriate popup messags on the map

In [17]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [18]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

Click the drive time areas to explore their attributes. Because the content of the pop-ups may include HTML source code, it is also possible to have the pop-up windows include other resources such as tables and images.

#### How long of a drive for tonight's party?

In [19]:
balboa_park = geocode('Balboa park, San Diego, CA')[0]
map1.draw(balboa_park['location'])

## Constructing service areas for different times of the day
The service areas for the facilities may look different depending on what time of day a vehicle would start driving. Therefore, we will run the solver using multiple day times for the `time_of_day` parameter to be able to compare visually the difference between the service areas. We will generate service areas for the following times: 6am, 10am, 2pm, 6pm, and 10pm.

In [ ]:
times = [datetime(2017, 6, 10, h).timestamp() * 1000 
         for h in [6, 10, 14, 18, 22]]

#loop through each time of the day and compute the service area
sa_results = []
for daytime in times:
    result = sa_layer.solve_service_area(facilities=fset, default_breaks=[5,10,15], 
                                         travel_direction='esriNATravelDirectionFromFacility',
                                         time_of_day=daytime, time_of_day_is_utc=False)
    sa_results.append(result)

The service area has been computed, we process it to generate a list of `FeatureSet` objects to animate on the map

In [ ]:
SD_fset_list=[]
for result in sa_results:
    poly_feat_list = []
    for polygon_dict in result['saPolygons']['features']:
        f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
        poly_feat_list.append(f1)
        
    service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])
    
    SD_fset_list.append(service_area_fset)

Draw and animate the results on a map

In [ ]:
map2= my_gis.map("San Diego, CA")
map2

In [ ]:
import time
map2.clear_graphics()

times = ['6 am', '10 am', '2 pm', '6 pm', '10 pm']
time.sleep(2)
loop = 3

for l in range(loop):
    map2.clear_graphics()
    j=0
    for fset in SD_fset_list:
        print(times[j])
        map2.draw(fset)
        j+=1
        time.sleep(1)

Thus from the animation above, we notice the service area is smallest at 6 AM and increases progressively later during the day.